<a href="https://colab.research.google.com/github/Whateveres/Desafios_Data_Science/blob/main/Desaf%C3%ADo_Manipulaci%C3%B3n_y_transformaci%C3%B3n_de_datos_(Parte_II)_Alejandra_Tobar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>Desafío - Manipulación y transformación de datos (Parte II)</h1></center><hr> 🚀
Nombre: Alejandra Tobar <hr>




**Descripción**

**Para desarrollar el desafío se usaron los siguientes archivos: US_Regional_Sales_Data.xlsx, reporte.xlsx**

**1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando pd.ExcelFile(archivo).sheet_names)**

In [ ]:
import pandas as pd

# Nombre del archivo Excel
archivo_excel = 'US_Regional_Sales_Data.xlsx'

# Nombres de las hojas en el archivo Excel
nombres_hojas = ['Sales Orders Sheet', 'Customers Sheet', 'Store Locations Sheet', 'Products Sheet', 'Sales Team Sheet']

# Creando un diccionario para almacenar los DataFrames
dataframes = {}

# Iterando sobre las hojas y leeyendo cada una en un DataFrame
for nombre_hoja in nombres_hojas:
    # Utilizar el nombre de la hoja como clave en el diccionario
    dataframes[nombre_hoja] = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

# Ahora, dataframes contiene los DataFrames correspondientes a cada hoja
# Puedo acceder a ellos utilizando los nombres de las hojas como claves
sales_orders_df = dataframes['Sales Orders Sheet']
customers_df = dataframes['Customers Sheet']
store_locations_df = dataframes['Store Locations Sheet']
products_df = dataframes['Products Sheet']
sales_team_df = dataframes['Sales Team Sheet']

sales_team_df

**2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado
en una nueva variable llamada df_base.**


In [67]:
# Uniendo tablas sales_orders y customers_df con método merge
df_base = sales_orders_df.merge(customers_df, on = '_CustomerID', validate = 'many_to_one', how ='inner')

# Uniendo tabla resultado y store_locations_df con método merge
df_base = df_base.merge(store_locations_df, on = '_StoreID', validate = 'many_to_one', how ='inner')

# Uniendo tabla resultado  y roducts_df con método merge
df_base = df_base.merge(products_df, on = '_ProductID', validate = 'many_to_one', how ='inner')

# Uniendo tabla resultado  y customers_df con método merge
df_base = df_base.merge(sales_team_df, on = '_SalesTeamID', validate = 'many_to_one', how ='inner')

#Exportando df_base a excel para revisión interna
df_base.to_excel('df_base.xlsx', index=False)

El método 'merge()' en Pandas se ha utilizado en este caso para combinar los DataFrames con la información de las hojas de datos. Se usó validate = 'many_to_one'

**3. Agrega las siguientes columnas a df_ base:**



In [ ]:
# Asegurandonos primero que las columnas de fecha son tipo datetime
df_base['OrderDate'] = pd.to_datetime(df_base['OrderDate'])
df_base['ProcuredDate'] = pd.to_datetime(df_base['ProcuredDate'])
df_base['ShipDate'] = pd.to_datetime(df_base['ShipDate'])
df_base['DeliveryDate'] = pd.to_datetime(df_base['DeliveryDate'])

# Agregando nuevas columnas y realizando cálculos
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

# Revisando actualizaciones de df_base
df_base.head()

**4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval que clasifique los valores de la columna CustomerDays en los siguientes intervalos:**

*   0 to 15 days
*   15 to 30 days
*   30 to 45 days
*   45 to 60 days
*   60 to 75 days
*   75 to 90 days

Hint: Utiliza pd.cut con estas etiquetas de Intervalo. Puedes convertir previamente la columna CustomerDays a valores numéricos con pd.to_numeric(). Considera que en este caso, el resultado es entregado en formato 10**9 segundos, por lo que debes dividir el resultado por este valor.

In [ ]:
# Convirtiendo columna CustomerDays a valores numéricos y dividiendo por 10**9
df_base['CustomerDays'] = pd.to_numeric(df_base['CustomerDays']) / 10**9

# Definiendo etiquetas de intervalos
intervalos = [0, 15, 30, 45, 60, 75, 90] # Definiendo los intervalos deseados
etiquetas = ['0 to 15 days', '15 to 30 days', '30 to 45 days', '45 to 60 days', '60 to 75 days', '75 to 90 days'] #Etiquetas para los intervalos

# Agregando nueva columna CustomerDaysInterval utilizando pd.cut
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins=intervalos, labels=etiquetas, right=False) #Valores cerrados al extremo derecho

# Mostrando resultados
print(df_base[['CustomerDays', 'CustomerDaysInterval']].head())


**5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales
Team en las filas y CustomerDaysInterval en las columnas. Este reporte debe
cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx**

In [ ]:
#Pivot Table
df_pivot = pd.pivot_table(
                                   data = df_base # DataFrame.
                                 , index = "Sales Team" # Columna del DataFrame original que queremos como fila/indice de la tabla resultante.
                                 , columns = "CustomerDaysInterval" #  Columna del DataFrame original que queremos como columna de la tabla resultante.
                                 , values = "OrderNumber" # valores a lo que le aplicaremos la aggfunc.
                                 , aggfunc= "count" # función a aplicar a la columna de valores (conteo en este caso).
                                 , fill_value = 0 # Valor para llenar los espacios vacíos en la tabla resultante.
)

#Imprimiendo Pivot Table
df_pivot.head()

**6. Agrega la siguiente columna calculada a la base:**

In [ ]:
# Agregando columna calculada GrossMargin a df_base
df_base['GrossMargin'] = df_base['Order Quantity'] * (df_base['Unit Price'] * (1 - df_base['Discount Applied']) - df_base['Unit Cost']).round(2)

df_base['GrossMargin'].head()

**7. En esta parte debes aplicar un porcentaje a la columna GrossMargin, pero ese
porcentaje dependerá del intervalo en que se encuentre GrossMargin.**

In [ ]:
# Tabla de comisiones
comisiones = {
    '0 to 100': 0.05,
    '100 to 1000': 0.10,
    '1000 to 10000': 0.15,
    '10000 to 100000': 0.20
}

# Aplicando pd.cut para obtener etiquetas numéricas
df_base['CommissionsInterval'] = pd.cut(df_base['GrossMargin'], bins=[0, 100, 1000, 10000, 100000], labels=False, right=False)

# Mapeando las etiquetas a porcentajes y creando la columna Commissions
df_base['Commissions'] = df_base['CommissionsInterval'].map(comisiones) * df_base['GrossMargin']

df_base.head()


**8. Calcula el monto de la comisión, utilizando la Tabla del Anexo.**

In [ ]:
# Calculando columna CommissionsAmount
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['Commissions']

# Mostrando DataFrame con nueva columna
print(df_base[['GrossMargin', 'Commissions', 'CommissionsAmount']].head())

**9. Calcula la comisión sobre el margen bruto.**

In [ ]:
# Calculando y creando columna NetMargin
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

# Mostrando DataFrame con nueva columna
print(df_base[['GrossMargin', 'CommissionsAmount', 'NetMargin']].head())

**10. Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y
CommissionAmount agrupando por Sales Team en las filas.**

In [ ]:
#Pivot Table 2
df_pivot_table = pd.pivot_table(
                                   data = df_base # DataFrame.
                                 , index = "Sales Team" # Columna del DataFrame original que queremos como fila/indice de la tabla resultante.
                                 , columns = ["GrossMargin","NetMargin","CommissionsAmount"] # Columnas del DataFrame original que queremos como columnas de la tabla resultante.
                                 , values = "OrderNumber" # valores a lo que le aplicaremos la aggfunc.
                                 , aggfunc= "sum" # función a aplicar a la columna de valores (suma en este caso).
                                 , fill_value = 0 # Valor para llenar los espacios vacíos en la tabla resultante.
)

#Imprimiendo Pivot Table
df_pivot_table.head()

---
🚀
Nombre: Alejandra Tobar <hr>
